<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

This notebook demonstrates the advances concepts of fairness detection using Indirect Bias mechanism using IBM Watson OpenScale.

We make use of the adult income dataset, where the attributes `age`, `sex` and `race` are not used to train the machine learning model that we are building as part of this notebook. But we will configure fairness against these attributes (specifically `age` and `sex`) to check if the model is indirectly behaving in a biased manner with these protected (sensitive) attributes.

## Provision services and configure credentials

If you have not already, provision an instance of IBM Watson OpenScale and an instance of IBM Watson Machine Learning using the Cloud catalog.

Your Cloud API key can be generated by going to the Users section of the Cloud console. From that page, click your name, scroll down to the API Keys section, and click Create an IBM Cloud API key. Give your key a name and click Create, then copy the created key and paste it below.

NOTE: You can also get OpenScale API_KEY using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: [Instructions](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

<b>How to get api key using console:</b>

<li> bx login --sso
<li> bx iam api-key-create 'my_key'

## Credentials for IBM Cloud services

### Retrieve your IBM Cloud API key

1.	From the IBM Cloud toolbar, click your Account name, such as <Your user name>’s Account.
1.	From the Manage menu, click Access (IAM).
1.	In the navigation bar, click IBM Cloud API keys.
1.	Click the Create an IBM Cloud API key button.
1.	Type a name and description and then click Save.
1.	Copy the newly created API key and paste it into your notebook in the following **CLOUD_API_KEY** code box, which is the first code box.

    Note: replace everything between the two sets of double quotation marks (").

In [ ]:
CLOUD_API_KEY = "***"

### Retrieve your Watson Machine Learning credentials

1.	Go to the IBM Cloud dashboard.
1.	In the Resource summary section, click Services.
1.	Click Machine Learning.
1.	In the navigation pane, click Service credentials.
1.	Click the New credential button.
1.	Copy your credentials by clicking the copy icon.
1.	Return to the notebook editor and update the credentials by replacing the sample credentials with your own in the below cell

   **Note**: You need to replace everything including the opening bracket ({) and the closing bracket (}).
   
   **IMPORTANT**: If you are reusing a WML instance that is already bound to Watson OpenScale. Please specify that instance credentials in `wml_credentials`

In [ ]:
# Enter WML credentials
WML_CREDENTIALS = {
  
}

In [ ]:
#Get details from COS credentials
COS_API_KEY_ID = "***"
COS_RESOURCE_CRN = "***" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003abfb5d29761c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"
COS_ENDPOINT = "***" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints

In [ ]:
BUCKET_NAME = "***" #example: "credit-risk-training-data"
FILE_NAME = "Indirect_bias_AdultCensusdata.csv"

In [ ]:
DB_CREDENTIALS = None
KEEP_MY_INTERNAL_POSTGRES = True
IAM_URL="https://iam.ng.bluemix.net/oidc/token"

# Package installation

The following opensource packages must be installed into this notebook instance so that they are available to use during processing.

In [ ]:
!rm -rf $PIP_BUILD
!pip install --upgrade watson-machine-learning-client --no-cache | tail -n 1
!pip install --upgrade ibm_watson_openscale --no-cache | tail -n 1
!pip install psycopg2-binary | tail -n 1
!pip install pyspark==2.3.0

Restart the kernel to assure the new libraries are being used.

# Load and explore data

In [ ]:
!rm Indirect_bias_AdultCensusdata.csv
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/notebooks/data/Indirect_bias_AdultCensusdata.csv

## Explore data

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
import json

spark = SparkSession.builder.getOrCreate()
df_data = spark.read.csv(path="Indirect_bias_AdultCensusdata.csv", sep=",", header=True, inferSchema=True) 
df = pd.read_csv("Indirect_bias_AdultCensusdata.csv")
df_data.head()

In [ ]:
print("Number of records: " + str(df_data.count()))

# Create a model

In [ ]:
# spark_df = sqlCtx.createDataFrame(df_data)
spark_df = df_data
# Remove protected attributes from training data
protected_attributes = ["race", "age", "sex"]
for attr in protected_attributes:
    spark_df = spark_df.drop(attr)
columns = spark_df.columns
model_name = "Adult Census Income Classifier Model"
deployment_name = "Adult Census Income Classifier Deployment"

spark_df.printSchema()

In [ ]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml import Pipeline, Model

cat_features = ['workclass', 'education', 'Marital', 'occupation', 'relationship', 'citizen_status'] 
num_features = ["fnlwgt", "education-num", "capitalgain", "loss", "hoursper"]
stages=[]

for feature in cat_features:
    string_indexer = StringIndexer(inputCol = feature, outputCol = feature + '_IX').setHandleInvalid("keep")
    encoder = OneHotEncoderEstimator(inputCols=[string_indexer.getOutputCol()], outputCols=[feature + "classVec"])
    stages += [string_indexer, encoder]

si_Label = StringIndexer(inputCol="label", outputCol="encoded_label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)
stages.append(si_Label)

In [ ]:
assembler_inputs = [c + "classVec" for c in cat_features] + num_features
va_features = VectorAssembler(inputCols=assembler_inputs, outputCol="features")
stages.append(va_features)

In [ ]:
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)
print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

In [ ]:
from pyspark.ml.classification import GBTClassifier, DecisionTreeClassifier, RandomForestClassifier
classifier = RandomForestClassifier(labelCol="encoded_label", featuresCol="features")
stages.append(classifier)
stages.append(label_converter)
pipeline = Pipeline(stages=stages)
model = pipeline.fit(train_data)

In [ ]:
predictions = model.transform(test_data)
predictions.printSchema()
predictions.head()

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluatorDT = BinaryClassificationEvaluator(labelCol="encoded_label", rawPredictionCol="rawPrediction")
accuracy = evaluatorDT.evaluate(predictions)

print("Accuracy = %g" % accuracy)

# Save and deploy the model

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import json

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

### Remove existing model and deployment

In [ ]:
model_deployment_ids = wml_client.deployments.get_uids()
for deployment_id in model_deployment_ids:
    deployment = wml_client.deployments.get_details(deployment_id)
    model_id = deployment['entity']['deployable_asset']['guid']
    if deployment['entity']['name'] == deployment_name:
        print('Deleting deployment id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting model id', model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

In [ ]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: "{}".format(model_name),
    wml_client.repository.ModelMetaNames.EVALUATION_METHOD: "binary",
    wml_client.repository.ModelMetaNames.EVALUATION_METRICS: [
        {
           "name": "accuracy",
           "value": accuracy,
           "threshold": 0.8
        }
    ]
}

In [ ]:
model_uid = None
model_deployment_ids = wml_client.deployments.get_uids()
for deployment_id in model_deployment_ids:
    deployment = wml_client.deployments.get_details(deployment_id)
    model_uid = deployment['entity']['deployable_asset']['guid']
    if deployment['entity']['name'] == deployment_name:
        print('Deleting existing deployment with id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting existing model with id', model_uid)
        wml_client.repository.delete(model_uid)

print("Storing model ...")

published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props, training_data=train_data, pipeline=pipeline)
model_uid = wml_client.repository.get_model_uid(published_model_details)
print("Done")

In [ ]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
for deployment in wml_deployments['resources']:
    if deployment_name == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']
        break

if deployment_uid is None:
    print("Deploying model...")

    deployment = wml_client.deployments.create(artifact_uid=model_uid, name=deployment_name, asynchronous=False)
    deployment_uid = wml_client.deployments.get_uid(deployment)
    
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

In [ ]:
cols_to_remove = ['label']
cols_to_remove.extend(protected_attributes)
cols_to_remove

## Create the meta data frame capturing the sensitive data

In [ ]:
meta_df = df[protected_attributes].copy()
meta_fields = meta_df.columns.tolist()
meta_values = meta_df[meta_fields].values.tolist()

## Construct the scoring payload comprising the meta fields

In [ ]:
def get_scoring_payload(no_of_records_to_score = 1):
    meta_payload = {
        "fields": meta_fields,
        "values": meta_values[:no_of_records_to_score]
    }

    for col in cols_to_remove:
        if col in df.columns:
            del df[col] 

    fields = df.columns.tolist()
    values = df[fields].values.tolist()

    payload_scoring = {"fields": fields,"values": values}

    payload_scoring = {
        "fields": fields,
        "values": values[:no_of_records_to_score],
        "meta": meta_payload
    }

    #import json
    #print(json.dumps(payload_scoring, indent=None))    
    return payload_scoring

## Method to perform scoring

In [ ]:
def payload_logging(no_of_records_to_score = 1):
    payload_scoring = get_scoring_payload(no_of_records_to_score)
    scoring_endpoint = None

    for deployment in wml_client.deployments.get_details()['resources']:
        if deployment_uid in deployment['metadata']['guid']:
            scoring_endpoint = deployment['entity']['scoring_url']

    print(scoring_endpoint)    
    scoring_response = wml_client.deployments.score(scoring_endpoint, payload_scoring)
    
    print('Single record scoring result:', '\n fields:', scoring_response['fields'], '\n values: ', scoring_response['values'][0])
    #print(json.dumps(scoring_response, indent=None))

## Score the model and print the scoring response

In [ ]:
payload_logging(no_of_records_to_score = 1)

# Configure OpenScale 

The notebook will now import the necessary libraries and set up a Python OpenScale client.

In [ ]:
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.utils import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.supporting_classes.enums import *

import json
import requests
import base64
from requests.auth import HTTPBasicAuth
import time

### Get Watson OpenScale GUID

Each instance of OpenScale has a unique ID. We can get this value using the Cloud API key specified at the beginning of the notebook.
1. Please update the `url` in the below WOS_CREDENTIALS payload as per the environment that you are using.
2. Please update the `DASHBOARD_URL` in the below cell as per the environment that you are using.

In [ ]:
from ibm_watson_openscale.utils import get_instance_guid
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator(
                    apikey=CLOUD_API_KEY,
                    url=IAM_URL
                )
WOS_GUID = get_instance_guid(authenticator)

WOS_CREDENTIALS = {
    "instance_guid": WOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}
DASHBOARD_URL = "https://aiopenscale.cloud.ibm.com"


if WOS_GUID is None:
    print('Watson OpenScale GUID NOT FOUND')
else:
    print(WOS_GUID)

In [ ]:
wos_client = APIClient(authenticator=authenticator, service_url=WOS_CREDENTIALS["url"])
wos_client.version

## Generate an IAM token
### The following is a function that will generate an IAM access token used to interact with the Watson OpenScale APIs

In [ ]:
def generate_access_token():
    headers={}
    headers["Content-Type"] = "application/x-www-form-urlencoded"
    headers["Accept"] = "application/json"
    auth = HTTPBasicAuth("bx", "bx")
    data = {
        "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
        "apikey": CLOUD_API_KEY
    }
    response = requests.post(IAM_URL, data=data, headers=headers, auth=auth)
    json_data = response.json()
    iam_access_token = json_data['access_token']
    return iam_access_token

## Create schema and datamart

### Set up datamart
Watson OpenScale uses a database to store payload logs and calculated metrics. If database credentials were not supplied above, the notebook will use the free, internal lite database. If database credentials were supplied, the datamart will be created there unless there is an existing datamart and the KEEP_MY_INTERNAL_POSTGRES variable is set to True. If an OpenScale datamart exists in Db2 or PostgreSQL, the existing datamart will be used and no data will be overwritten.

Prior instances of the Income classifier model will be removed from OpenScale monitoring.

In [ ]:
wos_client.data_marts.show()

In [ ]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print("Setting up external datamart")
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS["connection"]["postgres"]["hosts"][0]["hostname"],
                        username=DB_CREDENTIALS["connection"]["postgres"]["authentication"]["username"],
                        password=DB_CREDENTIALS["connection"]["postgres"]["authentication"]["password"],
                        db=DB_CREDENTIALS["connection"]["postgres"]["database"],
                        port=DB_CREDENTIALS["connection"]["postgres"]["hosts"][0]["port"],
                        ssl=True,
                        sslmode=DB_CREDENTIALS["connection"]["postgres"]["query_options"]["sslmode"],
                        certificate_base64=DB_CREDENTIALS["connection"]["postgres"]["certificate"]["certificate_base64"]
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print("Setting up internal datamart")
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print("Using existing datamart {}".format(data_mart_id))



In [ ]:
data_mart_details = wos_client.data_marts.list().result.data_marts[0]
data_mart_details.to_dict()

## Bind WML machine learning instance as Pre-Prod

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model. If a binding with name "WML Pre-Prod" already exists, this code will delete that binding a create a new one.

In [ ]:
wos_client.service_providers.show()

In [ ]:
service_providers = wos_client.service_providers.list().result.service_providers
existing_binding = False
for service_provider in service_providers:
    service_provider_id = service_provider.metadata.id
    if service_provider_id == WML_CREDENTIALS['instance_id']:
        existing_binding = True
        break

if not existing_binding:
    create_service_provider_result = wos_client.service_providers.add(
        name="Watson Machine Learning service provider",
        description="Service Provider added by tutorial WOS notebook",
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        credentials=WMLCredentialsCloud(
            apikey=WML_CREDENTIALS["apikey"],
            url=WML_CREDENTIALS["url"],
            instance_id=WML_CREDENTIALS["instance_id"]
        ),
        background_mode=False
    ).result
    service_provider_id = create_service_provider_result.metadata.id
else:
    print("Using existing service provider {}".format(service_provider_id))

In [ ]:
print(wos_client.service_providers.get(service_provider_id).result)

In [ ]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id, deployment_id=deployment_uid).result['resources'][0]
asset_deployment_details

## Patch binding as production

In [ ]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

payload = [
 {
   "op": "replace",
   "path": "/operational_space_id",
   "value": "production"
 }
]

In [ ]:
SERVICE_PROVIDER_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/service_providers/{1}".format(WOS_GUID, service_provider_id)
response = requests.patch(SERVICE_PROVIDER_URL, json=payload, headers=headers)
json_data = response.json()
print(json_data)

## Remove existing prod subscription

In [ ]:
wos_client.subscriptions.show()

### This code removes previous subscription that matches the model as it is expected this subscription is created only via this notebook.

In [ ]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_model_name = subscription.entity.asset.name
    if sub_model_name == model_name:
        wos_client.subscriptions.delete(subscription.metadata.id)
        print("Deleted existing subscription for model", model_uid)

In [ ]:
feature_columns = cat_features + num_features
feature_columns

#### This code creates the model subscription in OpenScale using the Python client API. Note that we need to provide the model unique identifier, and some information about the model itself.

In [ ]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=published_model_details["metadata"]["guid"],
            name=published_model_details["entity"]["name"],
            url=published_model_details["metadata"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details["metadata"]["guid"],
            name=asset_deployment_details["entity"]["name"],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details["metadata"]["url"]
        ),
        asset_properties=AssetPropertiesRequest(
            label_column="label",
            probability_fields=["probability"],
            prediction_field="predictedLabel",
            feature_fields = feature_columns,
            categorical_fields = cat_features,
            training_data_reference=TrainingDataReference(type="cos",
                                                          location=COSTrainingDataReferenceLocation(bucket = BUCKET_NAME,
                                                                                                    file_name = FILE_NAME),
                                                          connection=COSTrainingDataReferenceConnection.from_dict({
                                                                        "resource_instance_id": COS_RESOURCE_CRN,
                                                                        "url": COS_ENDPOINT,
                                                                        "api_key": COS_API_KEY_ID,
                                                                        "iam_url": IAM_URL})),
            training_data_schema=SparkStruct.from_dict(asset_deployment_details["entity"]["asset_properties"]["training_data_schema"])
        )
    ).result
subscription_id = subscription_details.metadata.id
print(subscription_details)

In [ ]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id:", payload_data_set_id)

In [ ]:
wos_client.subscriptions.show()

## Score the model so we can configure monitors

Now that the WML service has been bound and the subscription has been created, we need to send a request to the model before we configure OpenScale. This allows OpenScale to create a payload log in the datamart with the correct schema, so it can capture data coming into and out of the model. First, the code gets the model deployment's endpoint URL, and then sends a few records for predictions.

In [ ]:
payload_logging(no_of_records_to_score = 200)

In [ ]:
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    raise Exception("Payload logging did not happen!")

## Fairness, drift monitoring and explanations
###  Fairness configuration

The code below configures fairness monitoring for our model. It turns on monitoring for two features, sex and age. In each case, we must specify:

Which model feature to monitor One or more majority groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes One or more minority groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 80%) Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 100 records have been added. Finally, to calculate fairness, OpenScale must perform some calculations on the training data, so we provide the dataframe containing the data.

### Create Fairness Monitor Instance

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "features": [
        {
            "feature": "sex",
            "majority": ["Male"],
            "minority": ["Female"]
        },
        {
            "feature": "age",
            "majority": [[26,75]],
            "minority": [[18,25]]
        }
    ],
    "favourable_class": [">50K"],
    "unfavourable_class": ["<=50K"],
    "min_records": 200
}
thresholds = [
    {
        "metric_id": "fairness_value",
        "specific_values": [
            {
                "applies_to": [
                    {
                        "type": "tag",
                        "value": "sex",
                        "key": "feature"
                    }
                ],
                "value": 80
            },
            {
                "applies_to": [
                    {
                        "type": "tag",
                        "value": "age",
                        "key": "feature"
                    }
                ],
                "value": 80
            }
        ],
        "type": "lower_limit",
        "value": 80
    }
]
fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id

### Get Fairness Monitor Instance

In [ ]:
wos_client.monitor_instances.show()

### Get run details
In case of production subscription, initial monitoring run is triggered internally. Checking its status

In [ ]:
runs = wos_client.monitor_instances.list_runs(fairness_monitor_instance_id, limit=1).result.to_dict()
fairness_monitoring_run_id = runs["runs"][0]["metadata"]["id"]
run_status = None
while(run_status not in ["finished", "error"]):
    run_details = wos_client.monitor_instances.get_run_details(fairness_monitor_instance_id, fairness_monitoring_run_id).result.to_dict()
    run_status = run_details["entity"]["status"]["state"]
    print('run_status: ', run_status)
    if run_status in ["finished", "error"]:
        break
    time.sleep(30)

In [ ]:
FAIRNESS_DASHBOARD_URL = DASHBOARD_URL + "/aiopenscale/insights/{0}/fairness/age?features=fairnessv2,indirect_bias,v2transaction".format(deployment_uid)

In [ ]:
from IPython.display import Markdown as md
md("#### Link to IBM Watson OpenScale Fairness Dashboard: {}".format(FAIRNESS_DASHBOARD_URL))

### NOTE: Protected features with indirect bias will display proxy features contributing to indirect bias showing at the bottom of the fairness chart transaction details with their related correlation strengths.

### Run on-demand Fairness
If you would like to peform an on-demand fairness check, then we need to score a fresh set of data with meta-fields, so that they would be used for indirect bias checking. So the below two cells will score and make sure these records are reached to payload logging table.

In [ ]:
payload_logging(no_of_records_to_score = 200)

In [ ]:
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    raise Exception("Payload logging did not happen!")

### Trigger fairness monitoring run

In [ ]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id, background_mode=False)

### Check for its status

In [ ]:
time.sleep(10)

wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

In [ ]:
from IPython.display import Markdown as md
md("#### To view the latest evaluation of the fairness check, please visit IBM Watson OpenScale Fairness Dashboard: {}".format(FAIRNESS_DASHBOARD_URL))

## Additional data to help debugging

In [ ]:
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))
print("OpenScale Datamart id: {}".format(WOS_GUID))
print("OpenScale Subscription id: {}".format(subscription.metadata.id))
print("OpenScale Fairness Monitor Instance id: {}".format(fairness_monitor_instance_id))
print("OpenScale Fairness Monitoring Run id: {}".format(fairness_monitoring_run_id))
